# Chains in LangChain

## Introduction
This notebook demonstrates the `use of various chain mechanisms in LangChain` to manage and orchestrate complex workflows involving language models. The primary chain mechanisms covered include `LLMChain, SimpleSequentialChain, SequentialChain, and Router Chain`. These tools enable more effective and structured interactions with AI models by leveraging different chaining techniques. We will cover the following topics:     
* LLMChain  
* Sequential Chains  
  * SimpleSequentialChain  
  * SequentialChain  
* Router Chain  

### LLMChain
This section demonstrates how to use LLMChain to manage a single language model prompt and response.

### Setup

In [8]:
# Import necessary libraries
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Import necessary libraries
import os
from dotenv import load_dotenv, find_dotenv

# Load environment variables from a .env file
_ = load_dotenv(find_dotenv()) # read local .env file

**Note** To handle the deprecation of LLM models, we use the current date to select the appropriate model:

In [ ]:
# Handling Model Deprecation
import datetime

# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

- Ensure you have the required packages installed:
```py
%pip install pandas
```

In [ ]:
# Import the pandas library, commonly used for data manipulation and analysis
import pandas as pd

# Read the CSV file named 'Data.csv' into a DataFrame
df = pd.read_csv('Data.csv')

In [ ]:
# Display the first five rows of the DataFrame
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


### LLMChain

In [16]:
# Import LangChain-related libraries
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [17]:
# Initialize the language model
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [18]:
# Define a prompt template
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [19]:
#chain = LLMChain(llm=llm, prompt=prompt)
#product = 'Queen Size Sheet Set'
#chain.run(product)

In [20]:
# Create an LLMChain
chain = prompt | llm

In [ ]:
# Use .invoke() to get response
product = "Queen Size Sheet Set"
response = chain.invoke({"product": product})

response
# Print the response
print(response)

content='"Royal Linens"' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 24, 'total_tokens': 30, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-b7656615-8370-4fb4-9044-81aa2f45b16b-0' usage_metadata={'input_tokens': 24, 'output_tokens': 6, 'total_tokens': 30, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


### SimpleSequentialChain
This section demonstrates how to use SimpleSequentialChain to execute a sequence of chains in a simple linear fashion.

In [22]:
# Import SimpleSequentialChain
from langchain.chains import SimpleSequentialChain

In [23]:
# Initialize the language model
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# Define the first prompt template
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

C:\Users\Michela\AppData\Local\Temp\ipykernel_13772\3393886043.py:10: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain_one = LLMChain(llm=llm, prompt=first_prompt)


In [24]:
# Define the second prompt template
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [25]:
# Create a SimpleSequentialChain
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [ ]:
# Execute the chain
overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...
Regal Beddings Co.
Regal Beddings Co. offers luxurious and stylish bedding sets made from high-quality materials to elevate your bedroom décor.

> Finished chain.


{'input': 'Queen Size Sheet Set',
 'output': 'Regal Beddings Co. offers luxurious and stylish bedding sets made from high-quality materials to elevate your bedroom décor.'}

**Explanation output** The above output is an output of a process involving a SimpleSequentialChain in a language model. The output is essentially a log that shows the start and end of a sequential process (SimpleSequentialChain) and includes some processed or generated content related to Regal Beddings Co.

### SequentialChain
This section demonstrates how to use SequentialChain to manage more complex workflows with multiple chained operations.

In [28]:
# Import SequentialChain
from langchain.chains import SequentialChain

- Translate to English

In [29]:
# Initialize the language model
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# Define the first prompt template (translate to English)
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


- Summarize the review

In [30]:
# Define the second prompt template (summarize the review)
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


- Identify the language

In [31]:
# Define the third prompt template (identify the language)
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


- Write follow-up response

In [32]:

# Define the fourth prompt template (write follow-up response)
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


- Creation of the SequentialChain

In [33]:
# Create a SequentialChain
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [ ]:
# Execute the chain
review = df.Review[5]
overall_chain.invoke(review)




> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer finds the taste of the product mediocre and questions if it is an old batch or counterfeit compared to ones purchased in stores.',
 'followup_message': "Bonjour,\n\nNous vous remercions pour votre retour concernant le goût du produit. Nous nous excusons si vous avez trouvé la qualité médiocre. Notre priorité est de garantir la fraîcheur et l'authenticité de nos produits. Nous allons enquêter pour vérifier s'il s'agit d'un lot ancien ou contrefait. Votre satisfaction est importante pour nous et nous ferons tout notre possible pour rectifier la situation. N'hésitez pas à nous contacter si vous avez d'autres préoccup

**Explanation output** The output contains a detailed analysis and response to a customer's review. The original review in French expresses dissatisfaction with the product's taste and questions its authenticity. This review is translated into English, summarized, and a follow-up message is provided to address the customer's concerns and ensure their satisfaction.  
This structured approach helps in understanding customer feedback, translating it for broader accessibility, summarizing the key points for quick reference, and providing a professional and empathetic response to maintain customer relations.

### Router Chain
This section demonstrates how to use Router Chain to route inputs to different chains based on the content of the input.

In [35]:
# Define different prompts
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [ ]:
# Define a list of dictionaries containing information about 4 different subject prompts
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

- Import required modules from langchain

In [ ]:
# Import the MultiPromptChain class from the langchain.chains.router module
from langchain.chains.router import MultiPromptChain

# Import the LLMRouterChain class and RouterOutputParser from the langchain.chains.router.llm_router module
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser

# Import the PromptTemplate class from the langchain.prompts module
from langchain.prompts import PromptTemplate

In [ ]:
# Create an instance of ChatOpenAI with a specified temperature and model
llm = ChatOpenAI(temperature=0, model=llm_model)

In [ ]:
# Initialize an empty dictionary to store the destination chains
destination_chains = {}

# Loop through each prompt information dictionary in the prompt_infos list
for p_info in prompt_infos:

    # Extract the name and prompt template from the current prompt information dictionary
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    
    # Create a ChatPromptTemplate instance using the extracted prompt template
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    
    # Create an LLMChain instance with the initialized language model (llm) and the created prompt
    chain = LLMChain(llm=llm, prompt=prompt)
    
    # Add the created chain to the destination_chains dictionary with the name as the key
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
# Define the default prompt and chain
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
# Define the router template
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ "DEFAULT" or name of the prompt to use in {destinations}
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: The value of “destination” MUST match one of \
the candidate prompts listed below.\
If “destination” does not fit any of the specified prompts, set it to “DEFAULT.”
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.



<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
# Format the router template with destinations
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

# Create the router prompt
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

# Create the router chain
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
# Create the MultiPromptChain
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

C:\Users\Michela\AppData\Local\Temp\ipykernel_13772\3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [44]:
chain.run("What is black body radiation?")

C:\Users\Michela\AppData\Local\Temp\ipykernel_13772\1733274077.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run("What is black body radiation?")




> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This radiation is characterized by a continuous spectrum of wavelengths and intensities, with the peak intensity shifting to shorter wavelengths as the temperature of the black body increases. Black body radiation plays a key role in understanding concepts such as thermal radiation and the quantization of energy in quantum mechanics."

In [45]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

Issues in the above  Code  
- Deprecation Warning: MultiPromptChain is deprecated; use RouterChain instead.  
- Incorrect Prompt Formatting: You need to explicitly format MULTI_PROMPT_ROUTER_TEMPLATE with destinations.  


In [ ]:
# Import required modules from the langchain library
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda, RunnableBranch, RunnablePassthrough
from langchain.chains import LLMChain

# Define LLM
llm_model = "gpt-3.5-turbo"
llm = ChatOpenAI(temperature=0, model=llm_model)

# Define different prompts
physics_template = """You are a very smart physics professor...
Here is a question:
{input}"""

math_template = """You are a very good mathematician...
Here is a question:
{input}"""

history_template = """You are a very good historian...
Here is a question:
{input}"""

computerscience_template = """You are a successful computer scientist...
Here is a question:
{input}"""

# Define chains
chains = {
    "physics": LLMChain(llm=llm, prompt=ChatPromptTemplate.from_template(physics_template)),
    "math": LLMChain(llm=llm, prompt=ChatPromptTemplate.from_template(math_template)),
    "history": LLMChain(llm=llm, prompt=ChatPromptTemplate.from_template(history_template)),
    "computer science": LLMChain(llm=llm, prompt=ChatPromptTemplate.from_template(computerscience_template)),
}

# Default chain (fallback)
default_chain = LLMChain(llm=llm, prompt=ChatPromptTemplate.from_template("{input}"))

# Define routing function
def route_logic(inputs):
    input_text = inputs["input"].lower()
    if "physics" in input_text or "radiation" in input_text:
        return "physics"
    elif "math" in input_text or "calculate" in input_text:
        return "math"
    elif "history" in input_text:
        return "history"
    elif "code" in input_text or "computer" in input_text:
        return "computer science"
    return "default"

# Use RunnableBranch for routing
router = RunnableBranch(
    (lambda x: route_logic(x) == "physics", chains["physics"]),
    (lambda x: route_logic(x) == "math", chains["math"]),
    (lambda x: route_logic(x) == "history", chains["history"]),
    (lambda x: route_logic(x) == "computer science", chains["computer science"]),
    default_chain  # Fallback case
)

# Invoke the chain
response = router.invoke({"input": "What is black body radiation?"})
print(response)


C:\Users\Michela\AppData\Local\Temp\ipykernel_13772\4052776312.py:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, model=llm_model)


{'input': 'What is black body radiation?', 'text': "Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body is characterized by a continuous spectrum that depends only on the body's temperature, with the intensity of radiation increasing with increasing temperature. This phenomenon is described by Planck's law of black body radiation, which played a crucial role in the development of quantum mechanics."}


In [48]:
# Invoke the chain
response = router.invoke({"input": "Why does every cell in our body contain DNA?"})
print(response)

{'input': 'Why does every cell in our body contain DNA?', 'text': 'Every cell in our body contains DNA because DNA carries the genetic information that determines the characteristics and functions of an organism. DNA contains the instructions for building and maintaining an organism, including the proteins that are essential for cell structure and function. This genetic information is passed down from parent to offspring and is essential for the growth, development, and functioning of all cells in the body. Having DNA in every cell ensures that the genetic information is preserved and can be used to carry out the necessary processes for life.'}


## Conclusion
This notebook provides an `overview of different chain mechanisms in LangChain`, demonstrating how they can be used to manage and orchestrate complex workflows involving language models. By leveraging these chain tools, developers can create more structured and contextually aware interactions with AI models.